In [1]:
import torch
import os
import pymupdf
import transformers

In [2]:
pdf_path = "resume.pdf"
doc = pymupdf.open(pdf_path)


In [3]:
for page in doc:
    text = page.get_text()
    text = text.strip()
    print(text)

Senior Software Engineer at Infosys Technologies Aug 2019 –PresentIndianapolis, IN
 Successfully developed methodologies to streamline the process of 
data conversion leveraging latest tools
 Created optimized PL/SQL scripts to automate data gathering and 
transformation on various database technologies
 Refactored existing processes and built component to reduce 
manual efforts and performed unit tests
 Conducted meetings and workshops with client to document 
requirements and provide roadmap for the project
Java Developer at School of Education Jan –May 2019College Park, MD
 Successfully architectured and implemented features in Spring 
Boot to enhance the capabilities of  web application
 Designed and developed various UI pages in ReactJS to support 
application and contributed in creating process workflows
 Achieved a maintainable and optimised code by refactoring and 
following best coding and design practices
 Implemented unit tests in JUnit4 and Mockito providing full
co

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [104]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": f"""
        Please summarize the candidate's resume succinctly, noting their work experience, skills, projects and education. 
        in 5 sentences, Begin without any introduction. 
        The answer is to start with a summary, without a comment like Here's a summary. Highlight his experience, education, 
        projects and write a short summary of this cv like examples:

Example 1. The person graduated with a 1st degree in computer science, worked as a java developer from 1/12/2023 to 4/07/2024. 
He is skilled in technologies such as Java, Kotlin, Spring, HTML, CSS. The projects she has done are: 
Web application for checking stock charts, Website design for a game store.

Example 2 A person has completed a bachelor's degree in computer science and has not yet worked. 
She is skilled in technologies such as Python, Pytorch, Numpy, Pandas. The projects she has done are: 
Image classifier, Housing price analysis.

Enter the answer for this text: {text}
        """}]
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,  # Tokenizacja od razu
    return_dict=True,
    return_tensors="pt",  # teraz chcemy zwykły tekst, nie tokeny
).to(model.device)

#inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

response_only = output_text.split("model\n")[-1].strip()
print(response_only)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Here’s a 5-sentence summary of the candidate’s resume:

The candidate is a Senior Software Engineer with a strong background in Java and related technologies, demonstrated through experience at Infosys and School of Education.  They possess expertise in data transformation, PL/SQL scripting, and refactoring, alongside experience in Spring Boot and ReactJS development.  The candidate has also completed internships and a Master’s degree in Computer Engineering, focusing on software engineering, machine learning, and related fields.  Furthermore, they have demonstrated proficiency in various software management tools and programming languages, including JUnit and Postman.  Finally, the candidate’s passion for podcasting and machine learning highlights their broader interest in innovative technologies.


In [105]:
response_only



'Here’s a 5-sentence summary of the candidate’s resume:\n\nThe candidate is a Senior Software Engineer with a strong background in Java and related technologies, demonstrated through experience at Infosys and School of Education.  They possess expertise in data transformation, PL/SQL scripting, and refactoring, alongside experience in Spring Boot and ReactJS development.  The candidate has also completed internships and a Master’s degree in Computer Engineering, focusing on software engineering, machine learning, and related fields.  Furthermore, they have demonstrated proficiency in various software management tools and programming languages, including JUnit and Postman.  Finally, the candidate’s passion for podcasting and machine learning highlights their broader interest in innovative technologies.'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("data/UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  962 non-null    object
 1   Resume    962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [8]:
df.describe()

,Category,Resume
count,962,962
unique,25,166
top,Java Developer,"Technical Skills Web Technologies: Angular JS,..."
freq,84,18


In [9]:
df.isnull().sum()

Category    0
Resume      0
dtype: int64

In [10]:
df.isna().sum()

Category    0
Resume      0
dtype: int64

In [11]:
df["Resume"] = df["Resume"].str.replace("\r\n", "")


In [12]:
df["Resume"]

0      Skills * Programming Languages: Python (pandas...
1      Education Details May 2013 to May 2017 B.E   U...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details  MCA   YMCAUST,  Faridabad, ...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

In [13]:
df["Resume"][1]

'Education Details May 2013 to May 2017 B.E   UIT-RGPVData Scientist Data Scientist - MatelabsSkill Details Python- Exprience - Less than 1 year monthsStatsmodels- Exprience - 12 monthsAWS- Exprience - Less than 1 year monthsMachine learning- Exprience - Less than 1 year monthsSklearn- Exprience - Less than 1 year monthsScipy- Exprience - Less than 1 year monthsKeras- Exprience - Less than 1 year monthsCompany Details company - Matelabsdescription - ML Platform for business professionals, dummies and enthusiasts.60/A Koramangala 5th block,Achievements/Tasks behind sukh sagar, Bengaluru,India                               Developed and deployed auto preprocessing steps of machine learning mainly missing valuetreatment, outlier detection, encoding, scaling, feature selection and dimensionality reduction.Deployed automated classification and regression model.linkedin.com/in/aditya-rathore-b4600b146                           Reasearch and deployed the time series forecasting model ARIMA, S

In [14]:
import re
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text))

df['Resume'] = df['Resume'].apply(clean_text)

In [15]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab')
df['tokens'] = df['Resume'].apply(word_tokenize)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\piotr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
df.head()

,Category,Resume,tokens
0,Data Science,Skills Programming Languages Python pandas nu...,"[Skills, Programming, Languages, Python, panda..."
1,Data Science,Education Details May 2013 to May 2017 BE UI...,"[Education, Details, May, 2013, to, May, 2017,..."
2,Data Science,Areas of Interest Deep Learning Control System...,"[Areas, of, Interest, Deep, Learning, Control,..."
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA...,"[Skills, R, Python, SAP, HANA, Tableau, SAP, H..."
4,Data Science,Education Details MCA YMCAUST Faridabad H...,"[Education, Details, MCA, YMCAUST, Faridabad, ..."


In [17]:
from sentence_transformers import SentenceTransformer

In [18]:
from transformers import AutoTokenizer, Gemma3ForCausalLM

In [19]:
model_id = "sentence-transformers/all-mpnet-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

#tokens_num = tokenizer(df['tokens'], return_tensors="pt")

tokens = tokenizer(
    df['Resume'].tolist(),
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

In [20]:
df.head()

,Category,Resume,tokens
0,Data Science,Skills Programming Languages Python pandas nu...,"[Skills, Programming, Languages, Python, panda..."
1,Data Science,Education Details May 2013 to May 2017 BE UI...,"[Education, Details, May, 2013, to, May, 2017,..."
2,Data Science,Areas of Interest Deep Learning Control System...,"[Areas, of, Interest, Deep, Learning, Control,..."
3,Data Science,Skills R Python SAP HANA Tableau SAP HANA...,"[Skills, R, Python, SAP, HANA, Tableau, SAP, H..."
4,Data Science,Education Details MCA YMCAUST Faridabad H...,"[Education, Details, MCA, YMCAUST, Faridabad, ..."


In [21]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['labels']= label_encoder.fit_transform(df['Category'])

df['labels'].unique()

array([ 6, 12,  0,  1, 24, 16, 22, 14,  5, 15,  4, 21,  2, 11, 18, 20,  8,
       17, 19,  7, 13, 10,  9,  3, 23])

In [22]:
from sklearn.model_selection import train_test_split

In [27]:
categories = label_encoder.inverse_transform([5, 2, 11, 15, 6])
print(categories)

['Civil Engineer' 'Automation Testing' 'Electrical Engineering'
 'Java Developer' 'Data Science']


In [28]:
from torch import nn

In [29]:
tokens['input_ids'].shape

torch.Size([962, 256])

In [31]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device=device)

# Sentences are encode/embedded by calling model.encode()
embeddings = embedding_model.encode(df['Resume'])
# embeddings_dict = dict(zip(sentences, embeddings))
embeddings[:1]

array([[ 8.42282921e-02,  1.88610367e-02, -3.63459140e-02,
        -5.80457561e-02, -1.80319101e-02,  2.87969392e-02,
         2.27568690e-02,  3.61124948e-02, -2.05665361e-02,
        -7.18265399e-02,  1.84903610e-02,  4.56743836e-02,
         8.56274087e-03,  1.39780641e-01, -1.74210151e-03,
         1.52888549e-02,  1.65179837e-02,  3.36656943e-02,
         9.14636627e-03,  1.40186762e-02,  2.23156400e-02,
         3.80159989e-02,  3.51204327e-03,  2.69309841e-02,
        -9.66526289e-03,  9.40223504e-03,  1.30660105e-02,
        -3.26755531e-02, -2.07484625e-02, -4.83370423e-02,
         3.45595218e-02,  4.06770408e-02, -3.64129134e-02,
         2.91351639e-02,  2.76248647e-06, -3.50669660e-02,
        -4.29236256e-02, -3.27683315e-02, -5.78798167e-03,
        -2.02988298e-03,  6.38602674e-02, -1.75834298e-02,
         2.83530299e-02,  3.91860045e-02, -2.50785351e-02,
        -3.98297571e-02,  3.53656374e-02,  4.60782386e-02,
         3.96245383e-02,  4.09422964e-02, -1.85850840e-0

In [32]:
embeddings.shape

(962, 768)

In [35]:
df['labels'].values

array([ 6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 22

In [36]:
X, y = embeddings, df['labels'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [61]:
len(df['labels'].unique())

25

In [45]:
class CV_Classificator(nn.Module):
    def __init__(self, input_layer: int, hidden_layer: int, output_layer: int):
        super().__init__()

        self.seq = nn.Sequential(
            nn.Linear(in_features=input_layer,
                      out_features=hidden_layer),
            nn.ReLU(),
            nn.Linear(in_features=hidden_layer,
                      out_features=hidden_layer),
            nn.ReLU(),
            nn.Linear(in_features=hidden_layer,
                      out_features=output_layer)
        )

    def forward(self, x):
        x = self.seq(x)
        return x

model_cv = CV_Classificator(input_layer=768, hidden_layer=256, output_layer=len(df['labels'].unique()))
model_cv.to(device)

CV_Classificator(
  (seq): Sequential(
    (0): Linear(in_features=768, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=25, bias=True)
  )
)

In [46]:
model_cv.state_dict()

OrderedDict([('seq.0.weight',
              tensor([[ 0.0296, -0.0184, -0.0007,  ...,  0.0135, -0.0111, -0.0193],
                      [ 0.0235, -0.0070, -0.0021,  ..., -0.0104, -0.0291,  0.0147],
                      [-0.0008, -0.0103, -0.0221,  ..., -0.0307, -0.0045, -0.0055],
                      ...,
                      [-0.0331,  0.0180,  0.0034,  ..., -0.0270,  0.0246, -0.0210],
                      [ 0.0315, -0.0155, -0.0254,  ..., -0.0183,  0.0111,  0.0245],
                      [ 0.0069,  0.0194,  0.0214,  ..., -0.0114,  0.0114, -0.0012]],
                     device='cuda:0')),
             ('seq.0.bias',
              tensor([-8.9867e-03,  3.2230e-02, -1.2412e-02,  3.2205e-02,  1.6155e-02,
                      -1.5757e-02, -9.5822e-03, -9.0757e-03, -3.0210e-02,  1.2345e-02,
                       1.6377e-02,  2.6814e-02,  3.0337e-02,  1.0144e-02, -2.6017e-02,
                      -1.7719e-02, -2.9343e-02,  1.3652e-02,  2.3439e-02, -2.4102e-02,
                      